<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [1]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\kolov\Desktop\VNP\LABS\Lab3B\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<00:00, 16.4MB/s]


### Import the required libraries

In [2]:
import pandas as pd

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [3]:
data = pd.read_csv('ElectricCarData.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    object 
 1   Model            103 non-null    object 
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    object 
 7   RapidCharge      103 non-null    object 
 8   PowerTrain       103 non-null    object 
 9   PlugType         103 non-null    object 
 10  BodyStyle        103 non-null    object 
 11  Segment          103 non-null    object 
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 11.4+ KB


In [5]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
def label_data(data:pd.DataFrame, columns:list):
  encoder = LabelEncoder()
  data_copy = data.copy()

  for column in columns:
    data_copy[column] = encoder.fit_transform(data_copy[[column]].astype(str).values.ravel())
  return data_copy

In [8]:
data = label_data(data=data, columns=['Brand', 'Model','RapidCharge','PowerTrain','PlugType','BodyStyle','Segment','Seats'])

In [9]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,30,46,4.6,233,450,161,940,1,0,2,7,3,2,55480
1,31,33,10.0,160,270,167,250,1,2,2,1,2,2,30000
2,23,0,4.7,210,400,181,620,1,0,2,2,3,2,56440
3,2,101,6.8,180,360,206,560,1,2,2,6,3,2,68040
4,9,78,9.5,145,170,168,190,1,2,2,1,1,1,32997


In [10]:
import numpy as np

In [11]:
data.replace('-', np.nan, inplace=True)
data = data.apply(pd.to_numeric, errors='coerce')
data.fillna(data.mean(), inplace=True)

## Split the dataset for training and testing in ratio 80:20

In [12]:
input = data.copy()
input.drop(columns=['PriceEuro'],axis=1,inplace=True)
input.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats
0,30,46,4.6,233,450,161,940.0,1,0,2,7,3,2
1,31,33,10.0,160,270,167,250.0,1,2,2,1,2,2
2,23,0,4.7,210,400,181,620.0,1,0,2,2,3,2
3,2,101,6.8,180,360,206,560.0,1,2,2,6,3,2
4,9,78,9.5,145,170,168,190.0,1,2,2,1,1,1


In [13]:
target = data.copy()['PriceEuro']

In [14]:
target.head()

0    55480
1    30000
2    56440
3    68040
4    32997
Name: PriceEuro, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(input, target, test_size=0.2)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [17]:
from sklearn.tree import DecisionTreeRegressor

In [18]:
modelDT = DecisionTreeRegressor(max_depth=3, min_samples_split=5, max_features='sqrt')
modelDT.fit(X_train,Y_train)

DecisionTreeRegressor(max_depth=3, max_features='sqrt', min_samples_split=5)

## Predict the outcomes for X test

In [19]:
y_predDT = modelDT.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print("MAE:",mean_absolute_error(Y_test,y_predDT))
print("MSE:",mean_squared_error(Y_test,y_predDT))
print("R2:",r2_score(Y_test,y_predDT))

MAE: 14371.69074417646
MSE: 597229075.4270382
R2: 0.6516793407973129


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [21]:
from xgboost import XGBRegressor

In [22]:
modelXGB = XGBRegressor(max_depth=5, n_estimators=25, learning_rate=0.1, objective="reg:squarederror")
modelXGB.fit(X_train,Y_train)

C:\Users\kolov\miniconda3\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=25, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

## Predict the outcomes for X test

In [23]:
y_predXGB = modelXGB.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [24]:
print("MAE:",mean_absolute_error(Y_test,y_predXGB))
print("MSE:",mean_squared_error(Y_test,y_predXGB))
print("R2:",r2_score(Y_test,y_predXGB))

MAE: 9189.287109375
MSE: 352339904.0
R2: 0.7945055365562439


## Compare the performances of both model for at least three regression metircs

In [25]:
print("XGBRegressor")
print("MAE:",mean_absolute_error(Y_test,y_predXGB))
print("MSE:",mean_squared_error(Y_test,y_predXGB))
print("R2:",r2_score(Y_test,y_predXGB))
print("")
print("DecisionTreeRegressor")
print("MAE:",mean_absolute_error(Y_test,y_predDT))
print("MSE:",mean_squared_error(Y_test,y_predDT))
print("R2:",r2_score(Y_test,y_predDT))

XGBRegressor
MAE: 9189.287109375
MSE: 352339904.0
R2: 0.7945055365562439

DecisionTreeRegressor
MAE: 14371.69074417646
MSE: 597229075.4270382
R2: 0.6516793407973129
